# Colaboratoryで実行する場合
以下を実行して、外部ファイルをダウンロードしてください。   
**このセルはColaboratoryを起動するたびに必要となります**   
<font color="red">**自然言語処理のためのパッケージNLTKをアップグレードしています。以下のセルを実行後、ランタイムを再起動してください。**</font>

In [1]:
##################################
### Colaboratoryのみ以下を実行 ###
##################################
import sys
if 'google.colab' in sys.modules:
    !wget -P ./text http://www.hal.t.u-tokyo.ac.jp/~yamakata/lecture/mediaproc/mediaproc3/miyazawa_wakati.txt
    !pip install --upgrade nltk

# 自然言語処理２：言語モデル

Pythonプログラミング入門では、第３回の予習課題で文字n-gramを生成しました。   
今回は、形態素解析結果を用いて、単語n-gramを使った自然言語処理について勉強しましょう。

ここで、Janomeで生成した分かち書き文は単語区切りではなく形態素区切りですが、   
必ずしも言語学的な形態素とは言えない（単語と呼ぶべき）用語も交じっており、   
また、形態素区切りを単語区切りに変換するのは言語学的知識が必要となるので、   
ここではJanomeが出力した形態素を単語と呼び変えることにします。

本教材の作品データは[青空文庫](https://www.aozora.gr.jp/index.html)のものを使用しています。   
ただし、ルビや入力者注、アクセント分解された欧文や編者による注記等は削除しました。   
また、詩のように短い文章から構成されるものをのぞくなど、調整を行っています。


## 自然言語処理用パッケージNLTK
自然言語処理用のライブラリNLTK (Natural Language ToolKit)は、カレントバージョンのanacondaやColaboratoryではデフォルトインストールされているのでインストールは不要です。Colaboratoryの方は、このノートブックの冒頭のセルでバージョンアップを行っています。

## 1. 準備：分かち書き

言語モデルを学習する前に、解析したい文を分かち書きしましょう。

文章を分かち書きに変換するツールは前の資料で紹介しました。   
ここでは、宮沢賢治の作品147品の本文をjanomeを使って分かち書きし、１つのファイルにまとめた`miyazawa_wakati.txt`を使って処理を行います。   
（このファイルの作り方はわかりますね？本当は皆さんにやっていただきたいのですが、全147作品の形態素解析はそこそこ時間がかかります。）


In [25]:
file = 'text/miyazawa_wakati.txt'

# words= [(' ' + l + ' ').split() for l in open(file, 'r', encoding='utf-8').readlines()]
words = [('<BOP> ' + l + ' <EOP>').split() for l in open(file, 'r', encoding='utf-8').readlines()]

# 先頭の2文分を出力してみましょう
print(words[:2])
print(type(words))

[['<BOP>', 'その', '明け方', 'の', '空', 'の', '下', '、', 'ひる', 'の', '鳥', 'で', 'も', 'ゆか', 'ない', '高い', 'ところ', 'を', 'するどい', '霜', 'の', 'かけ', 'ら', 'が', '風', 'に', '流さ', 'れ', 'て', 'サラサラ', 'サラサラ', '南', 'の', 'ほう', 'へ', 'とん', 'で', 'ゆき', 'まし', 'た', '。', '<EOP>'], ['<BOP>', 'じつに', 'その', 'かすか', 'な', '音', 'が', '丘', 'の', '上', 'の', '一', '本', 'いちょう', 'の', '木', 'に', '聞こえる', 'くらい', 'すみきっ', 'た', '明け方', 'です', '。', '<EOP>']]
<class 'list'>


## 2. n-gram生成
### 2.1 自然言語処理用ライブラリNLTKの利用

n = 1のときuni-gram, n = 2のときbi-gram、n = 3のときtri-gramと呼びます。   
宮沢作品集の分かち書きが入っている`words`から、uni-gram, bi-gram, tri-gramをそれぞれ計算してみましょう。   
ここではNLTKのn-gram生成モジュールであるngramsを使います。

In [24]:
from nltk.util import ngrams

text_unigrams = [ngrams(word, 1) for word in words] # uni-gramを計算
text_bigrams = [ngrams(word, 2) for word in words] # bi-gramを計算
text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを計算

# 最初の1文のn-gramを出力してみましょう
print('unigram', [x for x in text_unigrams[0]])
print('bigram',[x for x in text_bigrams[0]])
print('trigram',[x for x in text_trigrams[0]])

unigram [('<BOP>',), ('その',), ('明け方',), ('の',), ('空',), ('の',), ('下',), ('、',), ('ひる',), ('の',), ('鳥',), ('で',), ('も',), ('ゆか',), ('ない',), ('高い',), ('ところ',), ('を',), ('するどい',), ('霜',), ('の',), ('かけ',), ('ら',), ('が',), ('風',), ('に',), ('流さ',), ('れ',), ('て',), ('サラサラ',), ('サラサラ',), ('南',), ('の',), ('ほう',), ('へ',), ('とん',), ('で',), ('ゆき',), ('まし',), ('た',), ('。',), ('<EOP>',)]
bigram [('<BOP>', 'その'), ('その', '明け方'), ('明け方', 'の'), ('の', '空'), ('空', 'の'), ('の', '下'), ('下', '、'), ('、', 'ひる'), ('ひる', 'の'), ('の', '鳥'), ('鳥', 'で'), ('で', 'も'), ('も', 'ゆか'), ('ゆか', 'ない'), ('ない', '高い'), ('高い', 'ところ'), ('ところ', 'を'), ('を', 'するどい'), ('するどい', '霜'), ('霜', 'の'), ('の', 'かけ'), ('かけ', 'ら'), ('ら', 'が'), ('が', '風'), ('風', 'に'), ('に', '流さ'), ('流さ', 'れ'), ('れ', 'て'), ('て', 'サラサラ'), ('サラサラ', 'サラサラ'), ('サラサラ', '南'), ('南', 'の'), ('の', 'ほう'), ('ほう', 'へ'), ('へ', 'とん'), ('とん', 'で'), ('で', 'ゆき'), ('ゆき', 'まし'), ('まし', 'た'), ('た', '。'), ('。', '<EOP>')]
trigram [('<BOP>', 'その', '明け方'), ('その', '明け方', 'の'), ('明け方', 'の', '

### 2.2 n-gramの出現回数

nltk.NgramCounterを使って各n-gramが宮沢作品集にそれぞれいくつずつ登場するか数えてみましょう。   
まずは下準備です。

In [30]:
from nltk.util import ngrams
from nltk.lm import NgramCounter

# じつはngramsが返すのはイテレータなので何度も使いまわせないことに注意！
text_unigrams = [ngrams(word, 1) for word in words] # uni-gramを計算
text_bigrams = [ngrams(word, 2) for word in words] # bi-gramを計算
text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを計算

# uni-gram, bi-gram, tri-gramをまとめて一気に出現回数を数える
ngram_counts = NgramCounter(text_unigrams + text_bigrams +text_trigrams) # N-gramの出現回数をカウント

In [32]:
print(type(ngram_counts))

<class 'nltk.lm.counter.NgramCounter'>


上で計算した結果を使ってさまざまなn-gramの出現回数を表示してみよう。   

ここで、
- `ngram_counts['ジヨバンニ'])`には「ジヨバンニ」の出現回数
- `ngram_counts[['ジヨバンニ']]`には、「ジヨバンニ」に続く単語の出現回数
- `ngram_counts[['ジヨバンニ','は']]`には、「ジヨバンニ は」という２つの単語の後に続く単語の出現回数

が記録されていることに注意してください。

In [33]:
# 「ジヨバンニ」のuni-gram出現回数
print('ジヨバンニ: ' + str(ngram_counts['ジヨバンニ']))

# 「ジヨバンニ」に続く単語の出現回数（bi-gram出現回数）
print('ジヨバンニ')
for word, count in sorted(ngram_counts[['ジヨバンニ']].items(), key=lambda x:x[1], reverse=True):
    print('->\t{:s}: {:d}'.format(word, count))
print()

# 「ジヨバンニ は」に続く単語の出現回数（tri-gram出現回数）
print('ジヨバンニ は')
for word, count in sorted(ngram_counts[['ジヨバンニ','は']].items(), key=lambda x:x[1], reverse=True):
    print('->\t{:s}: {:d}'.format(word, count))


ジヨバンニ: 205
ジヨバンニ
->	は: 123
->	が: 27
->	の: 21
->	も: 10
->	に: 6
->	、: 5
->	さん: 4
->	を: 3
->	たち: 3
->	と: 1
->	や: 1
->	まで: 1

ジヨバンニ は
->	、: 33
->	思は: 7
->	まるで: 5
->	思ひ: 5
->	まつ: 5
->	もう: 4
->	すぐ: 2
->	その: 2
->	何: 2
->	俄: 2
->	窓: 2
->	橋: 2
->	なんだか: 2
->	云: 2
->	また: 2
->	勢: 1
->	手: 1
->	拾: 1
->	おじぎ: 1
->	靴: 1
->	玄: 1
->	立つ: 1
->	高く: 1
->	われ: 1
->	帽子: 1
->	せ: 1
->	なんとも: 1
->	走り: 1
->	ぢ: 1
->	町: 1
->	眼: 1
->	一: 1
->	叫び: 1
->	まだ: 1
->	なぜ: 1
->	どんどん: 1
->	いきなり: 1
->	みんな: 1
->	わくわく: 1
->	かすか: 1
->	それ: 1
->	（: 1
->	胸: 1
->	びつくり: 1
->	困: 1
->	たしかに: 1
->	こんな: 1
->	首: 1
->	坊: 1
->	川下: 1
->	生: 1
->	熱: 1
->	どうしても: 1
->	だんだん: 1
->	もうす: 1
->	あぶなく: 1
->	そつ: 1
->	自分: 1
->	唇: 1
->	力強く: 1
->	叫ん: 1


## 3. 言語モデルの学習

n-gram確率とは、その文書において、直前の$(n-1)$単語が与えられたとき、その後に続く単語の出現確率を表しています。   
たとえば上の例では、「ジヨバンニ は」という２単語が現れたとき、その後に続くのは「、」が３３回です。   
「ジヨバンニ は」の出現回数は123回なので、「ジヨバンニ は」の後に「、」が続く確率（tri-gram確率）は$33/123=0.268293$ということになります。   
これをすべてのn-gramについて計算することを、ここでは「学習」と呼びます。

tri-gram確率を最尤推定法（Maximum Likelihood Estimation)により学習します。
これには少し時間がかかります。

In [21]:
from nltk.lm import Vocabulary
from nltk.lm.models import MLE
from nltk.util import ngrams

# 読み込んだ小説集の語彙（異なり単語）を収集
# Vocabularyは1次元のリストを受け取るが、wordsは2次元のリストなので、
# wordsを内包表記で2次元から1次元に変換してからVocabularyに渡しています
vocab = Vocabulary([item for sublist in words for item in sublist])

# 語彙の一覧を表示させたいなら下4行のコメントを有効にする
'''num = 0
for v in sorted(vocab.counts):
    print('{:d}\t{:s}'.format(num,v))
    num += 1
'''
print('Vocabulary size: ' + str(len(vocab))) # 語彙サイズ（単語の種類数）

text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを生成

n = 3
lm = MLE(order = n, vocabulary = vocab) # 最尤推定法（Maximum Likelihood Estimation)による統計的n-gram言語モデルの準備
lm.fit(text_trigrams) # 上で生成したtri-gramを使って言語モデルを学習

Vocabulary size: 22793


## 4. 統計的n-gramの活用
### 4.1 指定された単語列に続く単語を調べよう

宮沢作品において、「ジヨバンニは」と来たら次にはどんな単語が続くでしょうか？    
（「ジヨバンニ は」の後に「、」が続く確率は先ほど計算しましたね。0.268293でした）。  
その確率を計算してみましょう。   

In [7]:
context = ['ジヨバンニ', 'は']
#context = ['カムパネルラ', 'は']
print(context)

# contextに続く単語のリストを獲得
# lm.vocab.lookup(context)は、contextに含まれる単語のうち、vocabに含まれていない
# 単語があったとき、その単語を<UNK>に置き換える
# （UNKはUn-knownのこと。日本語では未知語と呼ぶ）
prob_list = [(word, lm.score(word, context)) for word
            in lm.context_counts(lm.vocab.lookup(context))]
prob_list.sort(key=lambda x: x[1], reverse=True)

sum_prob = 0.0 # 「ジョバンニは」に続く単語のtri-gram確率をすべて足すとどうなるでしょう？
for word, prob in prob_list:
    print('\t{:s}: {:f}'.format(word, prob))
    sum_prob += prob
    
print('ある単語列に続くtri-gram確率をすべて足したら', sum_prob) 
# 結果は1.0になります（誤差があります）
# このtri-gramが、『「ジヨバンニは」という単語列に続く』という条件のもと、
# それに続く単語の出現確率（つまり条件付き確率）になっていることが分かりますね。

['ジヨバンニ', 'は']
	、: 0.268293
	思は: 0.056911
	まるで: 0.040650
	思ひ: 0.040650
	まつ: 0.040650
	もう: 0.032520
	すぐ: 0.016260
	その: 0.016260
	何: 0.016260
	俄: 0.016260
	窓: 0.016260
	橋: 0.016260
	なんだか: 0.016260
	云: 0.016260
	また: 0.016260
	勢: 0.008130
	手: 0.008130
	拾: 0.008130
	おじぎ: 0.008130
	靴: 0.008130
	玄: 0.008130
	立つ: 0.008130
	高く: 0.008130
	われ: 0.008130
	帽子: 0.008130
	せ: 0.008130
	なんとも: 0.008130
	走り: 0.008130
	ぢ: 0.008130
	町: 0.008130
	眼: 0.008130
	一: 0.008130
	叫び: 0.008130
	まだ: 0.008130
	なぜ: 0.008130
	どんどん: 0.008130
	いきなり: 0.008130
	みんな: 0.008130
	わくわく: 0.008130
	かすか: 0.008130
	それ: 0.008130
	（: 0.008130
	胸: 0.008130
	びつくり: 0.008130
	困: 0.008130
	たしかに: 0.008130
	こんな: 0.008130
	首: 0.008130
	坊: 0.008130
	川下: 0.008130
	生: 0.008130
	熱: 0.008130
	どうしても: 0.008130
	だんだん: 0.008130
	もうす: 0.008130
	あぶなく: 0.008130
	そつ: 0.008130
	自分: 0.008130
	唇: 0.008130
	力強く: 0.008130
	叫ん: 0.008130
ある単語列に続くtri-gram確率をすべて足したら 1.0000000000000018


これはn-gram確率です。上でNgramCounterをつかって計算したn-gram出現回数と結果を比べてみてください。   
NgramCounterで数えたのは「回数」ですが、lm(言語モデル）では「確率」になっています。   

### 4.2 ランダム文生成

「ジヨバンニは」に続く文を適当に生成してみましょう。実際にあまり使いどころはないですが、今回の課題の一つですのでやってみてください。  

すでにtri-gram確率を計算しているので、ここでは、直前の2単語だけをみて、次に続く単語の出現確率が0じゃないものをランダムに選んでつないでいきます。   
まさしく人工無能の真骨頂です。

関数lm.generateは、text=seedで指定された単語列を直前の文として、次に続く単語をランダムに選びます。   
このとき、その単語が選ばれる確率は、そのn-gram確率に従います。

In [8]:
### ランダム文生成 ####
# contextから始まる文を生成

# 最初の2単語はいろいろと変えてみましょう
context = ['ジヨバンニ', 'は']
#context = ['カムパネルラ', 'は']
print(context)

for i in range(0, 100):
    # contextのうち最後の2単語から次に繋がる確率0じゃない単語をランダムに選ぶ
    w = lm.generate(text_seed=context)
    context.append(w) # 選ばれた単語をcontextに連結
    print(context)
    
    if '。' == w or '<EOP>' == w: # 句点「。」か<EOP>に到達したらそこで終了
        break
   

['ジヨバンニ', 'は']
['ジヨバンニ', 'は', 'もう']
['ジヨバンニ', 'は', 'もう', '、']
['ジヨバンニ', 'は', 'もう', '、', '硝子']
['ジヨバンニ', 'は', 'もう', '、', '硝子', 'で']
['ジヨバンニ', 'は', 'もう', '、', '硝子', 'で', 'こしらえ']
['ジヨバンニ', 'は', 'もう', '、', '硝子', 'で', 'こしらえ', 'た']
['ジヨバンニ', 'は', 'もう', '、', '硝子', 'で', 'こしらえ', 'た', 'もの']
['ジヨバンニ', 'は', 'もう', '、', '硝子', 'で', 'こしらえ', 'た', 'もの', '」']
['ジヨバンニ', 'は', 'もう', '、', '硝子', 'で', 'こしらえ', 'た', 'もの', '」', '<EOP>']


## 5. 文の生起確率$P(W)$の算出

宮沢賢治の小説集をコーパスとして学習した統計的tri-gramを使って、与えられた文（単語列）がどの程度『宮沢賢治らしい』かを推定してみましょう。   
これは、n-gram確率を

$$p(w_i|w_{i-n+1},...,w_{i-1})$$

としたとき、入力文$W=({w_0, w_1, ..., w_n})$に対して、文の生起確率

$$P(W)=\Pi_{i=0}^{n+1}p(w_i|w_{i-n+1},...,w_{i-1})$$

の計算をすることを意味しています(ただし、$w_i$はその文の$i$番目の単語を表し、$w_0$のとき文頭、$w_{n+1}$のとき文末記号を表す）。   
詳しくは授業でスライドを使って説明します。

In [9]:
### 入力文がどの程度宮沢賢治らしい文章かを判定してみよう ###

# どちらの文のほうが生起確率が高いでしょうか？構成する単語はどちらも同じです。
line = 'ジヨバンニ は 何 げ なく 答え まし た 。'
#line = '何 げ なく ジヨバンニ は 答え まし た 。'

words2 = line.split() # 空白で区切ってリストに代入します

n = 3 # lm (言語モデル) はtri-gramで学習しているので、ここではn=3を指定
probability = 1.0 # 始め確率は1.0に初期化
for ngram in ngrams(words2, n):
    # 2単語の後に3単語目が続く確率をひたすら計算
    prob = lm.score(lm.vocab.lookup(ngram[-1]),
                    lm.vocab.lookup(ngram[:-1]))
    print(ngram[:-1], ngram[-1], ':\t', prob)
    print()
    
    # 確率が0(すなわち、そのような文字の連結は宮沢作品に一度も現れない)の場合は
    # そのtri-gramの生起確率は0になってしまう。それをProbabilityに掛けると、
    # たとえ他のtri-gramが頻出するものであっても0になるので、
    # ここでは0のときは微小な値をprobに代入する
    prob = max(prob, 1e-8)
    
    # tri-gramの生起確率をかけ合わせていく
    probability *= prob
    
print(probability)

('ジヨバンニ', 'は') 何 :	 0.016260162601626018

('は', '何') げ :	 0.006493506493506494

('何', 'げ') なく :	 1.0

('げ', 'なく') 答え :	 0.3333333333333333

('なく', '答え') まし :	 1.0

('答え', 'まし') た :	 1.0

('まし', 'た') 。 :	 0.8878514702725907

3.124807201888539e-05


- 「何 げ なく ジヨバンニ は 答え まし た 。」の出現確率は6.658886027044431e-25
- 「ジヨバンニ は 何 げ なく 答え まし た 。」の出現確率は3.124807201888539e-05

ということで、後者の方がそれらしい文であることがわかります。